In [1]:
import numpy as np
from collections import defaultdict
import heapq

In [343]:
class Day16:
    def __init__(self, fname):
        with open(fname) as f:
            self.input = np.array([[item for item in line.strip()] for line in f.readlines()])
        mapping = {'E': 1, 'S': 1, '.': 1, '#' : 0}
        self.maze = np.array([[mapping[elem] for elem in row] for row in self.input])
        self.find_start_finish()
        self.optimal_cost, self.optimal_paths = self.dijkstra()

    def find_start_finish(self):
        for i, row in enumerate(self.input):
            for j, item in enumerate(row):
                if item == 'E':
                    self.dest = tuple([i, j])
                elif item == 'S':
                    self.src = tuple([i,j])
        

    def is_within_bounds(self, point):
        row, col = point
        num_rows, num_cols = self.input.shape
        return 0 <= row < num_rows and 0 <= col < num_cols


    def categorize_point(self, point):
        up = point + np.array([-1, 0])
        can_go_up = self.is_within_bounds(up) and self.input[tuple(up)] == '.'
        down = point + np.array([1, 0])
        can_go_down = self.is_within_bounds(down) and self.input[tuple(down)] == '.'
        right = point + np.array([0, 1])
        can_go_right = self.is_within_bounds(right) and self.input[tuple(right)] == '.'
        left = point + np.array([0, -1])
        can_go_left = self.is_within_bounds(left) and self.input[tuple(left)] == '.'
        if sum([can_go_up, can_go_down, can_go_left, can_go_right]) == 0:
            return "nothing"
        elif sum([can_go_up, can_go_down, can_go_left, can_go_right]) == 1:
            return "dead_end"
        elif (can_go_right | can_go_left) & (can_go_up | can_go_down):
            return "vertex"
        else:
            return "straight"

    def dijkstra(self):
        # Directions: right, down, left, up
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        rows, cols = len(self.maze), len(self.maze[0])
        
        # Priority queue: (cost, x, y, direction)
        pq = []
        heapq.heappush(pq, (0, self.src[0], self.src[1], 0, [self.src]))  # 0 indicates starting going east
        
        # Visited dictionary to track minimum costs: (x, y, direction) -> cost
        visited = {}
        optimal_paths = []
        optimal_cost = np.inf
        
        while pq:
            cost, x, y, prev_dir, path = heapq.heappop(pq)
            
            # If we reached the end position
            if (x, y) == self.dest:
                if cost == optimal_cost:
                    optimal_paths.append(path)
                elif cost < optimal_cost:
                    optimal_cost = cost
                    optimal_paths = [path]
                continue
            
            # Skip if we've already visited with a lower cost
            if (x, y, prev_dir) in visited:
                if cost > min(visited[(x, y, prev_dir)]):
                    continue
            else:
                visited[(x, y, prev_dir)] = {}
                visited[(x, y, prev_dir)][cost] = []
                visited[(x, y, prev_dir)][cost].append(path)
            
            # Explore neighbors
            for d, (dx, dy) in enumerate(directions):
                nx, ny = x + dx, y + dy
                
                # Check bounds
                if 0 <= nx < rows and 0 <= ny < cols:
                    move_cost = self.maze[nx][ny]
                    if move_cost == 0:  # 0 indicates an obstacle
                        continue
                    
                    # Calculate new cost
                    turn_penalty = 1000 if prev_dir != d else 0
                    new_cost = cost + move_cost + turn_penalty
                    new_path = path + [(nx, ny)]
                    
                    heapq.heappush(pq, (new_cost, nx, ny, d, new_path))
        
        return optimal_cost, optimal_paths  # Return -1 if no path is found

    def solve(self):
        optimal_spots = set()
        for path in self.optimal_paths:
            optimal_spots.update(set(path))
        return self.optimal_cost, len(optimal_spots)
        

In [344]:
Day16('data/test.txt').solve()

(7036, 45)

In [345]:

Day16('data/test2.txt').solve()

(11048, 64)

In [346]:
Day16('data/test3.txt').solve()

(4013, 14)

In [347]:
Day16('data/input.txt').solve()

(102504, 535)